# O problema do caxeiro viajante
Suponha que um caixeiro viajante tenha que visitar n cidades diferentes, iniciando e
encerrando sua viagem na primeira cidade. Suponha, também, que não importa a
ordem com que as cidades são visitadas e que de cada uma delas o caixeiro pode ir
diretamente para qualquer outra. O problema do caixeiro viajante consiste em descobrir
a rota que torna mínima a viagem total.

### O código a seguir implementa uma solução para o problema considerando que o caxeiro passe por 6 cidades

In [2]:
# Importação das bibliotecas utilizadas no problema
import numpy as np
import random
import itertools

### Geração das distâncias entre as cidades


In [3]:
# Geração aleatória das distâncias entre as cidades
num_cidades = 6
dist_max = 100

# Criação de uma matriz (6x6) de zeros
distancia = [[0] * num_cidades for _ in range(num_cidades)]

# Geração das distâncias
for i in range(num_cidades):
    for j in range(i + 1, num_cidades):
        distancia[i][j] = distancia[j][i] = random.randint(1, dist_max)

In [4]:
# Impressão em formato de  tabela
print("   ", end="")
for i in range(num_cidades):
    print(f"{i:2}", end=" ")
print()

for i in range(num_cidades):
    print(f"{i:2} ", end="")
    for j in range(num_cidades):
        print(f"{distancia[i][j]:2}", end=" ")
    print()


    0  1  2  3  4  5 
 0  0 98 86 38 13 86 
 1 98  0 80 97 58 99 
 2 86 80  0 20 33 71 
 3 38 97 20  0 91 55 
 4 13 58 33 91  0 82 
 5 86 99 71 55 82  0 


 ### Iniciar população de rotas


In [5]:
def inicializar_populacao(tamanho_populacao, num_cidades):
    populacao = []
    for _ in range(tamanho_populacao):
        rota = list(range(num_cidades))
        random.shuffle(rota)
        populacao.append(rota)
    return populacao

### Função de Fitness do problema
A função fitness definida para o problema calcula a distancia total da rota (contando o retorno para a cidade inicial) e retorna os valores negativos, pois a rota precisa ser minimizada


In [6]:
# A função fitness deve ser calculada como a menor distancia total
def funcao_fitness(rota):
  # Rota = indivíduo
  distancia_total = 0;
  for i in range(len(rota) - 1):
        distancia_total += distancia[rota[i]][rota[i + 1]]
  distancia_total += distancia[rota[-1]][rota[0]]  # Volte para a cidade inicial

  return -distancia_total


### Função de seleção por torneio

In [7]:
def selecao_torneio(fitnesses):
    # Índices dos indivíduos escolhidos no torneio
    ind1, ind2 = -1, -1

    # Verificação se não se trata dos mesmos indivíduos
    while ind1 == ind2:
        # Torneio 1
        # Escolha de 2 indivíduos aleatórios
        sorteados = random.sample(range(0, populacao_size-1), 2)
        print( "sorteados 1:", sorteados)
        # Escolha do índice do indivíduo com maior função de fitness
        if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
            ind1 = sorteados[0]
        else:
            ind1 = sorteados[1]

        # Torneio 2 - Mesmo processo anterior, agora retornando o índice ind2
        sorteados = random.sample(range(0, populacao_size-1), 2)
        print("sorteados 2:", sorteados)
        if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
            ind2 = sorteados[0]
        else:
            ind2 = sorteados[1]

    return ind1, ind2


### Função de crossover
Como o problema requer que os indivíduos não tenham genes repetidos (não pode passar mais de uma vez na mesma cidade)

In [26]:
def cruzamento(pai1, pai2):
    ponto_corte = random.randint(1, num_cidades -2)
    print("Ponto de corte:",ponto_corte)
    filho1 = [0] * num_cidades
    filho2 = [0] * num_cidades

#Inicia colocando os filhos com a primeira parte dos pais
    for i in range(ponto_corte):
      filho1[i] = pai1[i]
      filho2[i] = pai2[i]

#Começa a fazer o crossOver de modo a evitar a repetição
    i = ponto_corte
    while i < (num_cidades-1):
      for gene in pai2:
        if gene not in filho1:
          filho1[i] = gene
          i += 1

    i = ponto_corte
    while i < (num_cidades-1):
      for gene in pai1:
        if gene not in filho2:
          filho2[i] = gene
          i += 1

    return filho1,filho2

### Função de Elitismo
Função responsável por retornar os 2 índices de maiores fitness de uma população

In [14]:
def maxFit(fitnesses):

  id1 = fitnesses.index(max(fitnesses))
  fitnesses.pop(id1) #Não repetir
  id2 = fitnesses.index(max(fitnesses))

  return id1,id2

def minFit(fitnesses):

  id1 = fitnesses.index(min(fitnesses))
  fitnesses.pop(id1) #Não repetir
  id2 = fitnesses.index(min(fitnesses))

  return id1,id2


### Função de Mutação
Essa função de mutação troca 2 genes de lugar se um numero aleatório menor que 1 for maior que a taxa de mutação, ou seja, se aplicada, a mutação troca 2 genes de lugar em cada filho

In [10]:
def mutacao(filhos, taxa):
    for i in range(len(filhos)):
        if random.random() < taxa:
            pos1, pos2 = random.sample(range(len(filhos[i])), 2)
            filhos[i][pos1], filhos[i][pos2] = filhos[i][pos2], filhos[i][pos1]
    return filhos

# Algorítmo evolutivo

In [27]:
# Parametros iniciais
populacao_size = 50
taxa_mutacao = 0.2
num_geracoes = 10
taxa_cruzamento = 0.8
i = 0

# Inicialização da população
populacao_inicial = np.zeros(num_cidades, dtype=int)
populacao_inicial = inicializar_populacao(populacao_size, num_cidades)
populacao = populacao_inicial




#Algorítmo evolutivo
for geracao in range(num_geracoes):
  print(geracao+1,"Geração")
  # Iniciando a lista de valores de fitness
  fitness = [funcao_fitness(individuo) for individuo in populacao]

  #Impressão
  print("População no inicio da geração e fitness correspondente:")
  i = 0
  for individuo in populacao:
   print(i, individuo, fitness[i])
   i = i+1


  # Calculo dos 2 melhores individuos das geracao anterior (antes da selecao)
  elite = maxFit(fitness.copy()) # Pega os 2 melhores individuos da populacao anterior
  print("Índices dos melhores individuos da geração anterior: ", elite)

  #Seleção
  ind1, ind2 = selecao_torneio(fitness)

  # Cruzamento
  filho1,filho2 = cruzamento(populacao[ind1], populacao[ind2])
  print("Pais:", populacao[ind1], populacao[ind2])
  print("Filhos:",filho1,filho2)


  # Mutação
  filhos = mutacao((filho1,filho2), taxa_mutacao)
  print("Mutação:", filhos)



  # Substituição da população pelos filhos
  populacao[ind1], populacao[ind2] = filhos


  #Calculo dos piores da geração atual:
  fitness_novo = [funcao_fitness(individuo) for individuo in populacao]
  piores = minFit(fitness_novo.copy())
  print("Índices dos piores individuos da geração atual: ", piores)


  #Elitismo
  #Substituicao dos piores individuos da geracao atual pelos melhores da anterior
  populacao[piores[0]] = populacao[elite[0]]
  populacao[piores[1]] = populacao[elite[1]]


# Encontrando o melhor indivíduo
melhor_individuo = max(populacao, key=lambda x: funcao_fitness(x))

print("Melhor rota:", melhor_individuo)
print("Distância total:", -funcao_fitness(melhor_individuo))

1 Geração
População no inicio da geração e fitness correspondente:
0 [0, 3, 5, 2, 1, 4] -315
1 [5, 1, 0, 3, 2, 4] -370
2 [4, 2, 1, 5, 3, 0] -318
3 [2, 4, 5, 3, 1, 0] -451
4 [1, 0, 3, 5, 4, 2] -386
5 [4, 5, 1, 0, 2, 3] -476
6 [2, 3, 5, 4, 1, 0] -399
7 [5, 3, 4, 1, 0, 2] -459
8 [0, 1, 5, 3, 2, 4] -318
9 [4, 3, 2, 0, 1, 5] -476
10 [2, 5, 1, 4, 3, 0] -443
11 [0, 2, 1, 5, 4, 3] -476
12 [1, 3, 5, 2, 0, 4] -380
13 [0, 5, 2, 1, 3, 4] -438
14 [3, 4, 2, 1, 0, 5] -443
15 [0, 1, 2, 4, 5, 3] -386
16 [2, 3, 1, 5, 4, 0] -397
17 [3, 1, 2, 0, 5, 4] -522
18 [1, 4, 2, 0, 5, 3] -415
19 [3, 4, 2, 1, 5, 0] -427
20 [2, 3, 5, 1, 0, 4] -318
21 [4, 1, 5, 0, 2, 3] -440
22 [2, 3, 4, 5, 1, 0] -476
23 [1, 3, 2, 0, 4, 5] -397
24 [4, 2, 5, 1, 3, 0] -351
25 [5, 1, 4, 0, 2, 3] -331
26 [3, 2, 5, 0, 4, 1] -345
27 [4, 0, 2, 5, 3, 1] -380
28 [4, 3, 5, 1, 2, 0] -424
29 [3, 5, 1, 0, 4, 2] -318
30 [4, 3, 5, 2, 1, 0] -408
31 [2, 4, 5, 0, 3, 1] -416
32 [4, 3, 2, 1, 0, 5] -457
33 [5, 3, 0, 4, 1, 2] -315
34 [0, 5, 3, 4, 1, 2] -45